**Steps:**

1. Move to directory where you want to save the converted wiki-page
2. Run this script


**Parameters to customize for each page:**

```
- pagename 
- markup_format
```


**Pseudo-code**

```
  # using [Mediawiki API](https://www.mediawiki.org/wiki/API:Main_page)
  
  1. fetch wikitext from 'pagename'
  2. fetch list of images/files used in 'pagename'
  3. fetch all images/files in the list

  # using [pandoc](https://pandoc.org/)
  
  4. convert wikitext to 'markup_format'
```



In [ ]:
# the page to convert
# in this case 'xyz', where the page to convert is wiki.sugarlabs.org/go/Activities/xyz

# for example: when converting wiki.sugarlabs.org/go/Activities/Calculate
# pagename should be 'Calculate'
pagename = 'your-pagename-here'

# pandoc can deal with a lot of markup_formats
# I've used 1 for reStructuredText and anything-else for Markdown
markup_format = 'x'

In [ ]:
# Request is an easy-to-read HTTP library in Python
# Used to download wikitext and images
# Read: http://docs.python-requests.org/en/master/
import requests

# os provides a portable way of using operating system dependent functionality
# Used to Execute a command(string) in a subshell.
# Read: https://docs.python.org/3/library/os.html#os.system
import os

In [ ]:
# using mediawiki API to get the wikitext 
url = 'https://wiki.sugarlabs.org/api.php?action=parse&page=Activities/'+pagename+'&prop=wikitext&format=json'

In [ ]:
# GET request to url, reutrned value is stored in data
data = requests.get(url) 

In [ ]:
if data.status_code == 200:
    
    # storing result in json form
    json_obj = data.json()
    
    # fetching wikitext from json_obj
    wikitext = json_obj['parse']['wikitext']['*']
    
    # saving wikitext in a temporary file for conversion
    try:
        file_location = 'my-temp-wikitext-file'
        file = open(file_location,'w')
        file.write(wikitext)
        print('Wikitext Successfully written to file.')
    finally:
        file.close()

else:
    print('Sorry! Received status_code of GET request = {}'
          .format(data.status_code))

In [ ]:
# using pandoc to convert from mediawiki markup to markdown/reStructuredText
convertedfilename = pagename

if markup_format==1:
    convertedfileformat = '.rst'
    command = 'pandoc my-temp-wikitext-file -f mediawiki -t rst -s -o '+convertedfilename+convertedfileformat
else:
    convertedfileformat = '.md'
    command = 'pandoc my-temp-wikitext-file -f mediawiki -t markdown -s -o '+convertedfilename+convertedfileformat

os.system(command)

In [ ]:
#remove temporary wiki-text-file
command = 'rm my-temp-wikitext-file'
os.system(command)

In [ ]:
# url to get the images used in wiki-page
imageurl = 'https://wiki.sugarlabs.org/api.php?action=parse&page=Activities/'+pagename+'&prop=images&format=json'

In [ ]:
# getting data about images used
imagedata = requests.get(imageurl)

In [ ]:
# list of all images in wiki-page

if imagedata.status_code == 200:
    
    img_json_obj = imagedata.json()
    
    imagelist = img_json_obj['parse']['images']
    
    print('List of images in wiki:')
    print(*imagelist, sep='\n')

else:
    print('Sorry! Received status_code of GET request = {}'
          .format(imagedata.status_code))

In [ ]:
# downloading each individual image

for image in imagelist:
    
    try:
        # Read: https://www.mediawiki.org/wiki/API:Imageinfo
        image_info_url = 'https://wiki.sugarlabs.org/api.php?action=query&titles=File:'+image+'&prop=imageinfo&iiprop=url&format=json'
    
        # information about the image
        image_info = requests.get(image_info_url)
    
        # pages which satisfy given image condition in image_info_url
        imagepage = image_info.json()['query']['pages']
    
        #pageid for all pages in imagepage
        keys = []
        for k in imagepage:
            keys.append(k)
    
        for pageid in keys:
        
            # url of image in a given page-id
            imageurl = imagepage[pageid]['imageinfo'][0]['url']
            print(imageurl)
        
            # downloading the exact iamge
            imagefile = requests.get(imageurl)
        
            try:
                imagefile_loc = image
                file = open(imagefile_loc, 'wb')
                file.write(imagefile.content)
                print("File {} saved".format(image))
            finally:
                file.close()
        print()
    
    except:
        print("Failed to save {}".format(image))
        print()
    

In [ ]:
print('Conversion Complete')